In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# -----------------------------------------------------------
# 1. Load TRAIN and TEST datasets
# -----------------------------------------------------------
train_path = "Classification_Combined_Data/S1_S2_train_data.csv"
test_path  = "Classification_Combined_Data/S1_S2_test_data.csv"

df_train1 = pd.read_csv(train_path)
#df_test  = pd.read_csv(test_path)

# train only on ID = 5.0 fist 80%
df_train = df_train1[df_train1["ID"] == 18.0].sample(frac=0.8, random_state=42)
#test is last 20% of train
df_test = df_train1[df_train1["ID"] == 18.0].drop(df_train.index)

# # train only on ID = 5.0 fist 80%
# df_train = df_train1[df_train1["ID"] == 11.0].sample(frac=0.8, random_state=42)
# #test is last 20% of train
# df_test = df_train1[df_train1["ID"] == 11.0].drop(df_train.index)

#for both train and test, only rows where labsl is Not Drowsy or Slight
df_train = df_train[df_train["Label"].isin(["Not Drowsy", "Slight"])]
df_test = df_test[df_test["Label"].isin(["Not Drowsy", "Slight"])]
# -----------------------------------------------------------
# 2. Apply label mapping to both
# -----------------------------------------------------------
label_map = {
    'Not Drowsy': 'alert',
    'Slight': 'drowsy',
    'Moderate': 'drowsy',
    'Very': 'drowsy'
}

df_train["MappedLabel"] = df_train["Label"].map(label_map)
df_test["MappedLabel"]  = df_test["Label"].map(label_map)

# -----------------------------------------------------------
# 3. Encode target labels (alert=0, drowsy=1)
# -----------------------------------------------------------
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(df_train["MappedLabel"])
y_test  = label_encoder.transform(df_test["MappedLabel"])

# -----------------------------------------------------------
# 4. Select numeric features
# -----------------------------------------------------------
exclude_cols = ["Label", "MappedLabel", "ID", "Study"]
feature_cols = [c for c in df_train.columns if c not in exclude_cols]

X_train = df_train[feature_cols]
X_test  = df_test[feature_cols]

# -----------------------------------------------------------
# 5. Scale features (fit on train, transform on test)
# -----------------------------------------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# -----------------------------------------------------------
# 6. Fit unsupervised 2-component GMM on TRAIN DATA ONLY
# -----------------------------------------------------------
gmm = GaussianMixture(
    n_components=2,
    covariance_type='full',
    random_state=42
)
gmm.fit(X_train_scaled)

# -----------------------------------------------------------
# 7. Predict cluster labels on TRAIN (for alignment)
# -----------------------------------------------------------
train_cluster_labels = gmm.predict(X_train_scaled)

# -----------------------------------------------------------
# 8. Align cluster IDs to true labels using TRAIN accuracy
# -----------------------------------------------------------
acc0 = accuracy_score(y_train, train_cluster_labels)
acc1 = accuracy_score(y_train, 1 - train_cluster_labels)

# cluster → label mapping
if acc1 > acc0:
    cluster_to_label = lambda c: 1 - c
else:
    cluster_to_label = lambda c: c

# -----------------------------------------------------------
# 9. Predict on TEST
# -----------------------------------------------------------
test_clusters = gmm.predict(X_test_scaled)
test_preds = cluster_to_label(test_clusters)

# -----------------------------------------------------------
# 10. Evaluate TEST accuracy
# -----------------------------------------------------------
print("=== TEST SET RESULTS ===")
print("Accuracy:", accuracy_score(y_test, test_preds))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, test_preds))
print("\nClassification Report:")
print(classification_report(y_test, test_preds, target_names=label_encoder.classes_))

# -----------------------------------------------------------
# 11. Posterior probabilities on TEST set
# -----------------------------------------------------------
epsilon = 0.005 # to avoid exact 0 or 1 probabilities
probs_test = gmm.predict_proba(X_test_scaled)
# probs_test = np.clip(probs_test, epsilon, 1 - epsilon)
# probs_test = gmm.predict_proba(X_test_scaled)

df_test["GMM_prob_alert"] = probs_test[:, 0]
df_test["GMM_prob_drowsy"] = probs_test[:, 1]
df_test["GMM_pred_cluster"] = test_clusters
df_test["GMM_pred_label"] = label_encoder.inverse_transform(test_preds)

df_test.head(n=50)

=== TEST SET RESULTS ===
Accuracy: 0.676056338028169

Confusion Matrix:
[[14  9]
 [14 34]]

Classification Report:
              precision    recall  f1-score   support

       alert       0.50      0.61      0.55        23
      drowsy       0.79      0.71      0.75        48

    accuracy                           0.68        71
   macro avg       0.65      0.66      0.65        71
weighted avg       0.70      0.68      0.68        71



,window_start,ID,Study,Label,EAR_mean_mean,MAR_inner_mean,MAR_outer_mean,AU01_r_mean,AU15_r_mean,AU25_r_mean,...,gaze_angle_y_std,swAngle_std,laneDevPosition_std,laneDev_OffsetfrmLaneCentre_std,speed_std,MappedLabel,GMM_prob_alert,GMM_prob_drowsy,GMM_pred_cluster,GMM_pred_label
3866,1.711221e+09,18.0,S2,Not Drowsy,0.230844,0.045159,0.321186,2.332956,0.331031,0.220439,...,0.113499,0.265459,1.355525,1.016165,1.306433,alert,1.0,0.0,1,drowsy
3878,1.711221e+09,18.0,S2,Not Drowsy,0.259082,0.037471,0.322973,1.932660,0.152852,0.195459,...,0.111225,0.281281,0.000000,0.258603,0.274636,alert,1.0,0.0,1,drowsy
3885,1.711221e+09,18.0,S2,Not Drowsy,0.254160,0.035988,0.310708,2.463753,0.055803,0.284564,...,0.119449,0.319552,0.000000,0.557559,0.371462,alert,1.0,0.0,1,drowsy
3886,1.711221e+09,18.0,S2,Not Drowsy,0.260198,0.023953,0.309029,2.794150,0.104097,0.291275,...,0.066462,0.234215,0.000000,0.841261,0.302979,alert,1.0,0.0,1,drowsy
3899,1.711221e+09,18.0,S2,Slight,0.268915,0.018153,0.295259,3.379194,0.255431,0.032119,...,0.026566,1.624713,0.000000,0.665216,0.308420,drowsy,1.0,0.0,1,drowsy
3908,1.711221e+09,18.0,S2,Slight,0.233618,0.035544,0.322796,0.536794,0.277950,0.114972,...,0.058171,0.381156,1.196427,0.931916,0.482662,drowsy,1.0,0.0,1,drowsy
3913,1.711221e+09,18.0,S2,Slight,0.237888,0.049353,0.333951,0.049556,0.171800,0.433100,...,0.058046,1.394818,0.000000,0.713565,0.495197,drowsy,1.0,0.0,1,drowsy
3914,1.711221e+09,18.0,S2,Slight,0.242930,0.048705,0.335808,0.016511,0.171772,0.479572,...,0.052859,1.871865,1.101757,1.039753,0.171265,drowsy,1.0,0.0,1,drowsy
3915,1.711221e+09,18.0,S2,Slight,0.241957,0.043117,0.320149,0.017233,0.220617,0.428761,...,0.052697,0.448189,1.355266,0.815377,0.357664,drowsy,1.0,0.0,1,drowsy
3917,1.711221e+09,18.0,S2,Slight,0.239009,0.035954,0.265712,0.030011,0.740800,0.214278,...,0.058384,0.314605,1.354624,0.535909,0.218283,drowsy,1.0,0.0,1,drowsy


In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE

# ===========================================
# 1. LOAD TRAIN + TEST DATA
# ===========================================
train_path = "Classification_Combined_Data/S1_S2_train_data.csv"
test_path  = "Classification_Combined_Data/S1_S2_test_data.csv"

df_train1 = pd.read_csv(train_path)
#df_test  = pd.read_csv(test_path)

# # train only on ID = 5.0 fist 80%
# df_train = df_train1[df_train1["ID"] == 11.0].sample(frac=0.8, random_state=42)
# #test is last 20% of train
# df_test = df_train1[df_train1["ID"] ==11.0].drop(df_train.index)

# train only on ID = 5.0 fist 80%
df_train = df_train1[df_train1["ID"] == 18.0].sample(frac=0.8, random_state=42)
#test is last 20% of train
df_test = df_train1[df_train1["ID"] == 18.0].drop(df_train.index)

#for both train and test, only rows where labsl is Not Drowsy or Slight
df_train = df_train[df_train["Label"].isin(["Not Drowsy", "Slight"])]
df_test = df_test[df_test["Label"].isin(["Not Drowsy", "Slight"])]

# ===========================================
# 2. APPLY LABEL MAPPING
# ===========================================
label_map = {
    'Not Drowsy': 'alert',
    'Slight': 'drowsy',
    'Moderate': 'drowsy',
    'Very': 'drowsy'
}

df_train["MappedLabel"] = df_train["Label"].map(label_map)
df_test["MappedLabel"]  = df_test["Label"].map(label_map)

# ===========================================
# 3. ENCODE LABELS (alert=0, drowsy=1)
# ===========================================
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(df_train["MappedLabel"])
y_test  = label_encoder.transform(df_test["MappedLabel"])

# ===========================================
# 4. SELECT NUMERIC FEATURE COLUMNS
# ===========================================
exclude_cols = ["Label", "MappedLabel", "ID", "Study"]
feature_cols = [col for col in df_train.columns if col not in exclude_cols]

X_train = df_train[feature_cols]
X_test  = df_test[feature_cols]

# ===========================================
# 5. STANDARDIZE FEATURES (fit on train ONLY)
# ===========================================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# ===========================================
# 6. SMOTE on the TRAIN SET only
# ===========================================
sm = SMOTE(random_state=42)
X_train_bal, y_train_bal = sm.fit_resample(X_train_scaled, y_train)

print("Training balance after SMOTE:")
print(pd.Series(y_train_bal).value_counts())

# ===========================================
# 7. TRAIN SEPARATE GMMs (SUPERVISED)
#    One GMM for each class
# ===========================================
X_train_alert   = X_train_bal[y_train_bal == 0]
X_train_drowsy  = X_train_bal[y_train_bal == 1]

gmm_alert = GaussianMixture(
    n_components=2,
    covariance_type='full',
    random_state=42
)
gmm_drowsy = GaussianMixture(
    n_components=2,
    covariance_type='full',
    random_state=42
)

gmm_alert.fit(X_train_alert)
gmm_drowsy.fit(X_train_drowsy)

# ===========================================
# 8. CLASSIFICATION USING BAYES RULE
#    p(x | class) * P(class)
# ===========================================
# class priors from balanced training set
prior_alert  = (y_train_bal == 0).mean()
prior_drowsy = (y_train_bal == 1).mean()

# likelihoods from GMM
log_lik_alert  = gmm_alert.score_samples(X_test_scaled)
log_lik_drowsy = gmm_drowsy.score_samples(X_test_scaled)

# convert log-likelihoods + priors to posterior probabilities
log_posterior_alert  = log_lik_alert  + np.log(prior_alert)
log_posterior_drowsy = log_lik_drowsy + np.log(prior_drowsy)

# prediction: choose class with larger posterior
y_pred = np.where(log_posterior_alert > log_posterior_drowsy, 0, 1)

# ===========================================
# 9. EVALUATION
# ===========================================
print("\n=== TEST SET RESULTS ===")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# ===========================================
# 10. SAVE PROBABILITIES & PREDICTIONS
# ===========================================
# convert log posterior to normalized probabilities
posterior_alert = np.exp(log_posterior_alert)
posterior_drowsy = np.exp(log_posterior_drowsy)
posterior_sum = posterior_alert + posterior_drowsy

df_test["GMM_prob_alert"] = posterior_alert / posterior_sum
df_test["GMM_prob_drowsy"] = posterior_drowsy / posterior_sum
df_test["GMM_pred"] = y_pred
df_test["GMM_pred_label"] = label_encoder.inverse_transform(y_pred)

df_test.head()

Training balance after SMOTE:
1    200
0    200
Name: count, dtype: int64

=== TEST SET RESULTS ===
Accuracy: 0.676056338028169

Confusion Matrix:
[[ 0 23]
 [ 0 48]]

Classification Report:
              precision    recall  f1-score   support

       alert       0.00      0.00      0.00        23
      drowsy       0.68      1.00      0.81        48

    accuracy                           0.68        71
   macro avg       0.34      0.50      0.40        71
weighted avg       0.46      0.68      0.55        71



/Users/aaryannagpal1/Documents/Thesis/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/aaryannagpal1/Documents/Thesis/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/aaryannagpal1/Documents/Thesis/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(ave

,window_start,ID,Study,Label,EAR_mean_mean,MAR_inner_mean,MAR_outer_mean,AU01_r_mean,AU15_r_mean,AU25_r_mean,...,gaze_angle_y_std,swAngle_std,laneDevPosition_std,laneDev_OffsetfrmLaneCentre_std,speed_std,MappedLabel,GMM_prob_alert,GMM_prob_drowsy,GMM_pred,GMM_pred_label
3866,1.711221e+09,18.0,S2,Not Drowsy,0.230844,0.045159,0.321186,2.332956,0.331031,0.220439,...,0.113499,0.265459,1.355525,1.016165,1.306433,alert,0.0,1.0,1,drowsy
3878,1.711221e+09,18.0,S2,Not Drowsy,0.259082,0.037471,0.322973,1.932660,0.152852,0.195459,...,0.111225,0.281281,0.000000,0.258603,0.274636,alert,0.0,1.0,1,drowsy
3885,1.711221e+09,18.0,S2,Not Drowsy,0.254160,0.035988,0.310708,2.463753,0.055803,0.284564,...,0.119449,0.319552,0.000000,0.557559,0.371462,alert,0.0,1.0,1,drowsy
3886,1.711221e+09,18.0,S2,Not Drowsy,0.260198,0.023953,0.309029,2.794150,0.104097,0.291275,...,0.066462,0.234215,0.000000,0.841261,0.302979,alert,0.0,1.0,1,drowsy
3899,1.711221e+09,18.0,S2,Slight,0.268915,0.018153,0.295259,3.379194,0.255431,0.032119,...,0.026566,1.624713,0.000000,0.665216,0.308420,drowsy,0.0,1.0,1,drowsy


In [ ]:
# Keep classes separate for gmm
# alert vs slightly, or slightly vs moderate, or even alert vs moderate

# try changing parameters of gmm for sliding window approach
# maybe per minute change in consecutive labels
# try doing it by participant instead of all participants together
# state transitions from alert to slightly, or slightly to moderate
# See if there are newer gmm/hmm or dbscan approaches for better results